In [1]:
%load_ext rpy2.ipython

In [9]:
%%R

#install.packages("tibble")
#install.packages("dplyr")
#install.packages(ggplot2)
#install.packages("caret")
#install.packages("glmnet")

library(tibble)
library(dplyr)
library(ggplot2)
library(caret)
library(pROC)
library(ROSE)
library(glmnet)

In [11]:
%%R
# Convert BinaryDisengagement to a binary factor (0 = No Disengagement, 1 = Disengagement)
df$BinaryDisengagement <- as.factor(ifelse(df$BinaryDisengagement == "DISENGAGED", 1, 0))

# Verify the unique values
print(unique(df$BinaryDisengagement))

[1] 0
Levels: 0


In [12]:
%%R
print(unique(df$BinaryDisengagement))

[1] 0
Levels: 0


In [ ]:
%%R
df$BinaryDisengagement <- as.factor(ifelse(df$BinaryDisengagement %in% c("DISENGAGED", "1", "YES"), 1, 0))
df$BinaryDisengagement 

    [1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   [37] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   [73] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [109] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [145] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [181] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [217] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [253] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [289] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [325] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [361] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [397] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [433] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [3]:
%%R
# Define folder path
folder_path <- "/home/gampadu_linux/Desktop/TDMprivate/Preprocessed_Moving_Data/0sec_Window"

# List all CSV files in the directory (recursively)
file_list <- list.files(folder_path, pattern = "\\.csv$", full.names = TRUE, recursive = TRUE)

# Read all CSV files
df_list <- lapply(file_list, read.csv)

# Merge all data into one dataframe
df <- bind_rows(df_list)

# Convert to tibble for easy processing
df <- as_tibble(df)

# Check the structure of the final dataset
glimpse(df)

Rows: 11,302,633
Columns: 21
$ Ind                    <int> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1…
$ groupMetadataID        <chr> "01e65360-efd4-11ee-b966-fb353e7798cd", "01e653…
$ time                   <dbl> 1.69514e+18, 1.69514e+18, 1.69514e+18, 1.69514e…
$ NormalizedTime         <dbl> 0.000000e+00, 1.228293e-05, 3.373682e-05, 4.781…
$ BinaryDrivingMode      <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ BinaryDisengagement    <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ latitude               <dbl> 39.36440, 39.36440, 39.36440, 39.36440, 39.3644…
$ longitude              <dbl> -82.10334, -82.10334, -82.10334, -82.10334, -82…
$ ProgressAlongRoute     <dbl> 0.9998186, 0.9998186, 0.9998186, 0.9998186, 0.9…
$ solStatus              <chr> "SOL_COMPUTED", "SOL_COMPUTED", "SOL_COMPUTED",…
$ solType                <chr> "NARROW_INT", "NARROW_INT", "NARROW_INT", "NARR…
$ extendedSolutionStatus <int> 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,…
$ numSatsIn

In [4]:
%%R
# Defining GMIDs for Red Routes
# Am creating a list of GMIDs for processing
red_route_gmIDs  <- c(
     '61b12e7a-f234-11ee-bb33-fb353e7798cd',
'fcc6fcd2-f013-11ee-b966-fb353e7798cd',
'05c7c824-cab8-11ee-aa4d-1d66adf2f0c7',
'211bdb36-f0da-11ee-ba1b-fb353e7798cd',
'868de15e-f3b3-11ee-bb4e-fb353e7798cd',
'72a03d4a-efe9-11ee-b966-fb353e7798cd',
'7fb7b9c0-c881-11ee-a7fc-dd032dba19e8',
'96f7a614-f549-11ee-8afa-cb629b0d53e6',
'1bbbfbae-c839-11ee-a7fc-dd032dba19e8',
'622bd2e8-f0e4-11ee-ba1f-fb353e7798cd',
'9798fe24-f143-11ee-ba78-fb353e7798cd',
'88dd6fbe-f224-11ee-bb21-fb353e7798cd',
'c9c6856c-d33c-11ee-b437-336917683bb8',
'd12cd1c4-caec-11ee-909c-e1dc60cf66f9',
'817d6848-efb6-11ee-b966-fb353e7798cd',
'1b6aca0e-efdf-11ee-b966-fb353e7798cd',
'41b67a28-f52f-11ee-8afa-cb629b0d53e6',
'fc211bb2-efca-11ee-b966-fb353e7798cd',
'84d96f18-f214-11ee-bb13-fb353e7798cd',
'fe973c9c-f53c-11ee-8afa-cb629b0d53e6',
'3151e9e2-eff3-11ee-b966-fb353e7798cd',
'c0555ef0-f50f-11ee-8afa-cb629b0d53e6',
'f711e68e-f0e1-11ee-ba1f-fb353e7798cd',
'c338788a-d324-11ee-b437-336917683bb8',
'd21965e6-f0fa-11ee-ba37-fb353e7798cd',
'88a68dd8-eef9-11ee-9385-ef789ffde1d3',
'f41cbd44-eff8-11ee-b966-fb353e7798cd',
'43a1a35e-f362-11ee-bb4e-fb353e7798cd',
'6d2ea45a-c839-11ee-a7fc-dd032dba19e8',
'01e65360-efd4-11ee-b966-fb353e7798cd',
'7cbd932e-f244-11ee-bb3f-fb353e7798cd',
'65cfbfd6-f396-11ee-bb4e-fb353e7798cd',
'94c53148-eeed-11ee-9385-ef789ffde1d3',
'fd1ab258-efa7-11ee-b966-fb353e7798cd',
'e7b934a8-ef1a-11ee-9385-ef789ffde1d3',
'de933de8-f112-11ee-ba4d-fb353e7798cd',
'd3698592-ef9d-11ee-b966-fb353e7798cd',
'dd72fdec-f0cf-11ee-ba0d-fb353e7798cd',
'b82476fe-f1f3-11ee-baff-fb353e7798cd',
'f755cf60-f132-11ee-ba6d-fb353e7798cd',
'853ef120-cad3-11ee-909c-e1dc60cf66f9',
'f0eebb6a-f0dc-11ee-ba1e-fb353e7798cd',
'cf831f42-f353-11ee-bb4e-fb353e7798cd',
'ecebb942-f162-11ee-ba97-fb353e7798cd',
'219f7eb8-ef87-11ee-b966-fb353e7798cd',
'd24820c8-f197-11ee-babe-fb353e7798cd',
'3d2d29ec-ef95-11ee-b966-fb353e7798cd',
'457dc5ee-f02a-11ee-b966-fb353e7798cd',
'2462c9d0-eecd-11ee-9385-ef789ffde1d3',
'51ef6da6-ca9f-11ee-909c-e1dc60cf66f9',
'c25271be-f3a4-11ee-bb4e-fb353e7798cd',
'8347b862-efad-11ee-b966-fb353e7798cd',
'8dbbbf1c-f0ef-11ee-ba29-fb353e7798cd',
'9189a2a8-f121-11ee-ba5b-fb353e7798cd',
'8fa6fe80-c869-11ee-a7fc-dd032dba19e8',
'35518ec4-f153-11ee-ba88-fb353e7798cd',
'2f95c748-f009-11ee-b966-fb353e7798cd',
'8437f77a-cab7-11ee-909c-e1dc60cf66f9',
'2a61b8a8-f528-11ee-8afa-cb629b0d53e6',
'5a4bccf4-effe-11ee-b966-fb353e7798cd'
)

In [5]:
%%R
# Check if selected GMIDs exist in the dataset
existing_gmIDs <- unique(df$groupMetadataID)

# Identify GMIDs that are present in the dataset
valid_gmIDs <- red_route_gmIDs[red_route_gmIDs %in% existing_gmIDs]

# Identify GMIDs that are missing
missing_gmIDs <- red_route_gmIDs[!(red_route_gmIDs %in% existing_gmIDs)]

# Print results
print(paste("Valid GMIDs found in dataset:", length(valid_gmIDs)))
print(valid_gmIDs)

if (length(missing_gmIDs) > 0) {
    print(paste("Missing GMIDs (not found in dataset):", length(missing_gmIDs)))
    print(missing_gmIDs)
} else {
    print("All selected GMIDs are present in the dataset.")
}

[1] "Valid GMIDs found in dataset: 58"
 [1] "61b12e7a-f234-11ee-bb33-fb353e7798cd"
 [2] "fcc6fcd2-f013-11ee-b966-fb353e7798cd"
 [3] "05c7c824-cab8-11ee-aa4d-1d66adf2f0c7"
 [4] "211bdb36-f0da-11ee-ba1b-fb353e7798cd"
 [5] "868de15e-f3b3-11ee-bb4e-fb353e7798cd"
 [6] "72a03d4a-efe9-11ee-b966-fb353e7798cd"
 [7] "7fb7b9c0-c881-11ee-a7fc-dd032dba19e8"
 [8] "96f7a614-f549-11ee-8afa-cb629b0d53e6"
 [9] "1bbbfbae-c839-11ee-a7fc-dd032dba19e8"
[10] "622bd2e8-f0e4-11ee-ba1f-fb353e7798cd"
[11] "9798fe24-f143-11ee-ba78-fb353e7798cd"
[12] "88dd6fbe-f224-11ee-bb21-fb353e7798cd"
[13] "c9c6856c-d33c-11ee-b437-336917683bb8"
[14] "d12cd1c4-caec-11ee-909c-e1dc60cf66f9"
[15] "817d6848-efb6-11ee-b966-fb353e7798cd"
[16] "1b6aca0e-efdf-11ee-b966-fb353e7798cd"
[17] "41b67a28-f52f-11ee-8afa-cb629b0d53e6"
[18] "84d96f18-f214-11ee-bb13-fb353e7798cd"
[19] "fe973c9c-f53c-11ee-8afa-cb629b0d53e6"
[20] "3151e9e2-eff3-11ee-b966-fb353e7798cd"
[21] "c0555ef0-f50f-11ee-8afa-cb629b0d53e6"
[22] "f711e68e-f0e1-11ee-ba1f-fb353e7

In [8]:
%%R
# Load required libraries
library(ROSE)
library(dplyr)

# Step 1: Retain `groupMetadataID` and create a unique row index
df <- df %>% mutate(row_id = row_number())

# Step 2: Keep only the necessary columns for balancing
df_subset <- df %>% select(row_id, BinaryDisengagement, speedMps, 
                           throttlePercentage, brakePercentage, 
                           steeringPercentage, LatLonTotalStdDev)

# Step 3: Apply ROSE for balancing, ensuring `row_id` is preserved
df_balanced <- ROSE(BinaryDisengagement ~ speedMps + throttlePercentage + 
                    brakePercentage + steeringPercentage + LatLonTotalStdDev, 
                    data = df_subset, seed = 123)$data

# Step 4: Ensure `row_id` remains numeric (ROSE may convert it to a factor)
df_balanced$row_id <- as.integer(df_balanced$row_id)

# Step 5: Restore `groupMetadataID` using `row_id`
df_balanced <- df_balanced %>% left_join(df %>% select(row_id, groupMetadataID), by = "row_id")

# Step 6: Remove rows where `groupMetadataID` is missing
df_balanced <- df_balanced %>% filter(!is.na(groupMetadataID))

# Step 7: Check if `groupMetadataID` is successfully restored
print(unique(df_balanced$groupMetadataID))
print(paste("Valid GMIDs in dataset:", length(unique(df_balanced$groupMetadataID))))

Error in `$<-.data.frame`(`*tmp*`, row_id, value = integer(0)) : 
  replacement has 0 rows, data has 11302633


RInterpreterError: Failed to parse and evaluate line '# Load required libraries\nlibrary(ROSE)\nlibrary(dplyr)\n\n# Step 1: Retain `groupMetadataID` and create a unique row index\ndf <- df %>% mutate(row_id = row_number())\n\n# Step 2: Keep only the necessary columns for balancing\ndf_subset <- df %>% select(row_id, BinaryDisengagement, speedMps, \n                           throttlePercentage, brakePercentage, \n                           steeringPercentage, LatLonTotalStdDev)\n\n# Step 3: Apply ROSE for balancing, ensuring `row_id` is preserved\ndf_balanced <- ROSE(BinaryDisengagement ~ speedMps + throttlePercentage + \n                    brakePercentage + steeringPercentage + LatLonTotalStdDev, \n                    data = df_subset, seed = 123)$data\n\n# Step 4: Ensure `row_id` remains numeric (ROSE may convert it to a factor)\ndf_balanced$row_id <- as.integer(df_balanced$row_id)\n\n# Step 5: Restore `groupMetadataID` using `row_id`\ndf_balanced <- df_balanced %>% left_join(df %>%

In [ ]:
%%R
# Ensure there are enough GMIDs
if (length(red_route_gmIDs) >= 60) {

    # Split GMIDs into Training (80%) and Testing (20%)
    training_gmIDs <- red_route_gmIDs[1:55]
    testing_gmIDs <- red_route_gmIDs[56:60]

    # Filter df for training and testing data based on GMIDs
    train_data <- df %>% filter(groupMetadataID %in% training_gmIDs)
    test_data <- df %>% filter(groupMetadataID %in% testing_gmIDs)

    # Check the number of rows in each dataset
    print(paste("Training data rows:", nrow(train_data)))
    print(paste("Testing data rows:", nrow(test_data)))

    # Verify if test_data is not empty
    if (nrow(test_data) == 0) {
      print("Error: test_data is empty. Check GMID filtering.")
    }
} else {
    print("Error: Not enough GMIDs available for proper train-test split.")
}

In [ ]:
%%R
# Train a Logistic Regression Model
model <- glm(
  BinaryDisengagement ~ speedMps + throttlePercentage + brakePercentage + steeringPercentage + LatLonTotalStdDev,
  data = train_data,
  family = binomial
)

# Display model summary
summary(model)

In [ ]:
%%R
# Train a Logistic Regression Model to Predict BinaryDisengagement
model <- glm(
  BinaryDisengagement ~ speedMps + throttlePercentage + brakePercentage + steeringPercentage + LatLonTotalStdDev,
  data = df,
  family = binomial
)

# View Model Summary
summary(model)

In [ ]:
%%R
# Predict on the test set
predictions <- predict(model, test_data, type = "response")

# Convert probabilities to binary classes (0 or 1)
predicted_classes <- ifelse(predictions > 0.5, 1, 0)

# Calculate accuracy
accuracy <- mean(predicted_classes == test_data$BinaryDisengagement)

# Print the accuracy
print(paste("Model Accuracy:", accuracy))

In [ ]:
%%R
# Ensure predicted_classes only contains 0 and 1
predicted_classes <- ifelse(predicted_classes > 0.5, 1, 0)

# Convert it to a factor
predicted_classes <- as.factor(predicted_classes)

# Check again
print(unique(predicted_classes))  # Should return only 0 and 1

In [ ]:
%%R
table(train_data$BinaryDisengagement)
table(test_data$BinaryDisengagement)

In [ ]:
%%R
print(unique(test_data$BinaryDisengagement))

In [ ]:
%%R
summary(predictions)

In [ ]:
%%R
predicted_classes <- ifelse(predictions > 0.3, 1, 0)

In [ ]:
%%R
summary(model)

In [ ]:
%%R
# Load required libraries
library(ggplot2)
library(caret)

# Map the values to labels (1 -> "Disengagement", 0 -> "No Disengagement")
predicted_classes_labeled <- factor(predicted_classes, levels = c(0,1), labels = c("No Disengagement(0)", "Disengagement(1)"))
actual_classes_labeled <- factor(test_data$BinaryDisengagement, levels = c(0,1), labels = c("No Disengagement(0)", "Disengagement(1)"))

# Generate the confusion matrix
cm <- confusionMatrix(predicted_classes_labeled, actual_classes_labeled)
cm_table <- as.data.frame(cm$table)

# Rename columns for better understanding
colnames(cm_table) <- c("Actual", "Predicted", "Frequency")

# Create the colored confusion matrix plot
ggplot(cm_table, aes(x = Predicted, y = Actual, fill = Frequency)) +
  geom_tile(color = "white") +
  scale_fill_gradient(low = "lightblue", high = "blue", name = "Frequency") +
  geom_text(aes(label = Frequency), color = "white", size = 5) +
  labs(title = "Confusion Matrix", x = "Predicted Classes", y = "True Classes") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5, size = 14),
        axis.title

In [ ]:
%%R
# Check the structure of test_data
print(head(test_data$BinaryDisengagement))
print(unique(test_data$BinaryDisengagement))  # Ensure only 0 and 1 exist

# Check structure of predicted_classes
print(head(predicted_classes))
print(unique(predicted_classes))  # Ensure only 0 and 1 exist